In [2]:
import os
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
import json
import audio_metadata

##### Download all audio clips first!!!!

#### Importing dataset

In [ ]:
audio_clips_df = pd.read_csv('./csvs/modified_audio_clips_xx.csv', low_memory=False)      # last updated audio clips collection

In [ ]:
audio_clips_df.info()

#### Retrieve audio metadata - file_size_bytes

In [ ]:
def clip_file_size(audio_file):
    wave_file = audio_metadata.load(audio_file)
    filesize = wave_file['filesize']
    return filesize

In [ ]:
# Download all audio clips to find audio metadata
audio_clips_file_size_dict = {}
filepath = os.environ.get('AUDIO_CLIPS_DIRECTORY_PATH')

for index, audio_clip in audio_clips_df.iloc[:].iterrows():
    print("index: ", index)
    
    if audio_clip['file_name'] == 'extended.mp3':
        continue
    
    if audio_clip['file_name']:
        # directory path where all audio clips are stored
        audio_file = filepath + audio_clip['file_name']
        filesize = clip_file_size(audio_file)
        audio_clips_file_size_dict[audio_clip['_id']] = filesize
        
    else:
        filesize = None
        audio_clips_file_size_dict[audio_clip['_id']] = filesize

In [ ]:
file_size_bytes = open('./youtube_api/audio_clips_file_size_bytes.txt',"a")
file_size_bytes.write(json.dumps(audio_clips_file_size_dict))
file_size_bytes.close()

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to audio clips collection

In [ ]:
audio_clips = database['modified_audio_clips_xx']   # last updated audio clips collection

#### Update audio clips duration

In [ ]:
try:
    for (key,value) in audio_clips_file_size_dict.items():
        audio_clips.find_one_and_update(
        {"_id" : ObjectId(key)},
        {"$set": { "file_size_bytes" : value}}
    );
except Exception as e:
    print("Exception: ", e)